<a href="https://colab.research.google.com/github/fiorellaguillen/CASA0025/blob/main/notebooks/W04_postgis2_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [ ]:
%pip install duckdb duckdb-engine jupysql
%pip install leafmap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72

In [ ]:
import duckdb
import pandas as pd
import leafmap

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 72.5MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [ ]:
%sql duckdb:///nyc_data.db


In [ ]:
%%sql

INSTALL spatial;
LOAD spatial;

,Success


In [ ]:
%sql SELECT * FROM ( SHOW TABLES);

,name
0,nyc_census_blocks
1,nyc_homicides
2,nyc_neighborhoods
3,nyc_streets
4,nyc_subway_stations


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [ ]:
%%sql

SELECT * FROM nyc_subway_stations LIMIT 5;

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geom
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."


In [ ]:
%%sql

SELECT  subway.name, subway.routes
FROM nyc_subway_stations AS subway
JOIN nyc_neighborhoods AS neighborhoods
ON ST_Contains(neighborhoods.geom, subway.geom)
WHERE neighborhoods.name = 'Little Italy';

,NAME,ROUTES
0,Spring St,6


2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [ ]:
%%sql

SELECT DISTINCT neighborhoods.name
FROM nyc_neighborhoods AS neighborhoods
JOIN nyc_subway_stations AS subway
ON ST_Contains(neighborhoods.geom, subway.geom)
WHERE subway.routes LIKE '%6%';

,NAME
0,Chinatown
1,Greenwich Village
2,Murray Hill
3,Midtown
4,Upper East Side
5,Yorkville
6,Hunts Point
7,South Bronx
8,Soundview
9,Parkchester


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [ ]:
%%sql

SELECT * FROM nyc_neighborhoods LIMIT 5;

,BORONAME,NAME,geom
0,Brooklyn,Bensonhurst,"[5, 4, 41, 0, 0, 0, 0, 0, 54, 71, 14, 73, 198,..."
1,Manhattan,East Village,"[5, 4, 152, 0, 0, 0, 0, 0, 35, 215, 14, 73, 13..."
2,Manhattan,West Village,"[5, 4, 91, 0, 0, 0, 0, 0, 161, 95, 14, 73, 212..."
3,The Bronx,Throggs Neck,"[5, 4, 141, 0, 0, 0, 0, 0, 128, 232, 17, 73, 1..."
4,The Bronx,Wakefield-Williamsbridge,"[5, 4, 126, 0, 0, 0, 0, 0, 83, 85, 17, 73, 17,..."


4. **What neighborhood has the highest population density (persons/km2)?**


In [ ]:
%%sql

SELECT nyc_neighborhoods.NAME, SUM(nyc_census_blocks.POPN_TOTAL) / ST_AREA(nyc_neighborhoods.geom) / 1000000 AS density
FROM nyc_neighborhoods
JOIN nyc_census_blocks
ON ST_INTERSECTS(nyc_neighborhoods.geom, nyc_census_blocks.geom)
GROUP BY nyc_neighborhoods.NAME, nyc_neighborhoods.geom
ORDER BY density DESC
LIMIT 1;

,NAME,density
0,North Sutton Area,6.843513e-08


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [1]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.2 MB/s eta 0:00:00


In [2]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [3]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [4]:
%%sql

SELECT * FROM "https://storage.googleapis.com/qm2/casa0025_ships.csv" ;

,vesselid,vessel_name,vsl_descr,dwt,v_length,draught,sog,date,lat,lon,geom
0,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,5.2,2022-07-25 02:53:29,45.151777,36.513327,POINT (36.5133266666667 45.1517766666667)
1,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:09:37,45.146487,36.520780,POINT (36.52078 45.1464866666667)
2,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:13:58,45.146218,36.521965,POINT (36.521965 45.1462183333333)
3,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.1,2022-07-25 04:16:06,45.145058,36.522020,POINT (36.52202 45.1450583333333)
4,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.0,2022-07-25 05:20:17,45.144933,36.521848,POINT (36.5218483333333 45.1449333333333)
...,...,...,...,...,...,...,...,...,...,...,...
101323,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,0.1,2022-08-10 14:16:47,45.091987,36.522157,POINT (36.5221566666667 45.0919866666667)
101324,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,0.1,2022-08-10 14:43:48,45.091643,36.522213,POINT (36.5222133333333 45.0916433333333)
101325,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,5.8,2022-08-10 15:04:28,45.100457,36.519397,POINT (36.5193966666667 45.1004566666667)
101326,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,8.3,2022-08-23 06:06:51,45.087527,36.506987,POINT (36.5069866666667 45.0875266666667)


In [13]:
%%sql

DROP TABLE IF EXISTS ais;

CREATE TABLE ais AS (
   SELECT
      vesselid,
      date,
      sog,
      ST_TRANSFORM(ST_GEOMFROMTEXT(geom), 'EPSG:4326', 'EPSG:3857') AS geom
FROM "https://storage.googleapis.com/qm2/casa0025_ships.csv"
WHERE sog < 1
);

CREATE INDEX ais_index ON ais USING RTREE(geom);

SELECT * FROM ais LIMIT 5;

,vesselid,date,sog,geom
0,350053,2022-07-25 03:09:37,0.7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,350053,2022-07-25 03:13:58,0.7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,350053,2022-07-25 04:16:06,0.1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,350053,2022-07-25 05:20:17,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,350053,2022-07-25 06:23:57,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [14]:
%%sql

DROP TABLE IF EXISTS vinfo;

CREATE TABLE vinfo AS
SELECT DISTINCT vesselid, vessel_name, vsl_descr, dwt, v_length
FROM "https://storage.googleapis.com/qm2/casa0025_ships.csv"
WHERE sog < 1
LIMIT 1000;

SELECT * FROM vinfo LIMIT 5;

,vesselid,vessel_name,vsl_descr,dwt,v_length
0,256543,Omskiy 143,general cargo,3104.0,108.0
1,296750,Omskiy 99,general cargo,3108.0,54.0
2,286084,Omskiy- 119,general cargo,3157.0,108.0
3,278778,Omskiy-103,general cargo,3283.0,108.0
4,232730,Omskiy-138,general cargo,3104.0,108.0


In [15]:
%%sql
SELECT COUNT(*) FROM ais


,count_star()
0,86140


In [16]:
%%sql
SELECT COUNT(*) FROM vinfo

,count_star()
0,782


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [ ]:
%%sql

SELECT
a1.vesselid AS ship1,
a2.vesselid AS ship2,
a1.date AS start,
a2.date AS end

FROM ais AS a1
JOIN ais AS a2
ON ST_DWITHIN(a1.geom, a2.geom, 500)
AND ABS(EXTRACT(EPOCH FROM (a2.date - a1.date))) > 7200
AND a1.vesselid < a2.vesselid


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))